<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [108]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [109]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.18.1
Pandas:  1.0.1


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [0]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [60]:
#ANSWER:Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:
bikes=pd.read_table(r'C:/Users/Travel Your Style/Documents/IOD Course 22 June 20/DATA/bikeshare.csv', header=0, sep=",")
print(bikes.head())

   instant      dteday  season  yr  mnth  hr  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1   0        0        6           0   
1        2  2011-01-01       1   0     1   1        0        6           0   
2        3  2011-01-01       1   0     1   2        0        6           0   
3        4  2011-01-01       1   0     1   3        0        6           0   
4        5  2011-01-01       1   0     1   4        0        6           0   

   weathersit  temp   atemp   hum  windspeed  casual  registered  cnt  
0           1  0.24  0.2879  0.81        0.0       3          13   16  
1           1  0.22  0.2727  0.80        0.0       8          32   40  
2           1  0.22  0.2727  0.80        0.0       5          27   32  
3           1  0.24  0.2879  0.75        0.0       3          10   13  
4           1  0.24  0.2879  0.75        0.0       0           1    1  


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [4]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [13]:
df = pd.read_excel('C:/Users/Travel Your Style/Documents/IOD Course 22 June 20/DATA/Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [16]:
#ANSWER
df.drop(df.columns[[0]],axis=1)


,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,0.2,1.4,3.5,5.1,Setosa
1,0.2,1.4,3.0,4.9,Setosa
2,0.2,1.3,3.2,4.7,Setosa
3,0.2,1.5,3.1,4.6,Setosa
4,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...
145,2.3,5.2,3.0,6.7,Verginica
146,1.9,5.0,2.5,6.3,Verginica
147,2.0,5.2,3.0,6.5,Verginica
148,2.3,5.4,3.4,6.2,Verginica


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [18]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2020-07-05T03:16:27.840Z,35.650167,-117.4855,9.71,2.57,ml,30,64,0.0663,0.12,...,2020-07-05T03:34:33.040Z,"15km SSW of Searles Valley, CA",earthquake,0.21,0.46,0.165,26,automatic,ci,ci


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [19]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name           City  ST   CERT  \
 0                 The First State Bank  Barboursville  WV  14361   
 1                   Ericson State Bank        Ericson  NE  18265   
 2     City National Bank of New Jersey         Newark  NJ  21111   
 3                        Resolute Bank         Maumee  OH  58317   
 4                Louisa Community Bank         Louisa  KY  58112   
 ..                                 ...            ...  ..    ...   
 556                 Superior Bank, FSB       Hinsdale  IL  32646   
 557                Malta National Bank          Malta  OH   6629   
 558    First Alliance Bank & Trust Co.     Manchester  NH  34264   
 559  National State Bank of Metropolis     Metropolis  IL   3815   
 560                   Bank of Honolulu       Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                         MVB Bank, Inc.      April 3, 2020  
 1             Farmers and Merchants Bank  F

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [11]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [20]:
#ANSWER
bikes.shape


(17379, 17)

What are the column names in `bikes`?

In [21]:
#ANSWER
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [24]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [25]:
#ANSWER
bikes.index

RangeIndex(start=0, stop=17379, step=1)

https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [28]:
#ANSWER
# yes
bikes['season']


0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64

In [29]:
bikes[['season']]

,season
0,1
1,1
2,1
3,1
4,1
...,...
17374,1
17375,1
17376,1
17377,1


How would we use object notation to show the first 4 rows of `atemp`?

In [31]:
#ANSWER
bikes.atemp[:4]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [34]:
bikes[[0]]
#E: bikes[0]
#E: bikes[0,0]
#E: bikes[[0,0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [37]:
#ANSWER
bikes[:1]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [43]:
#ANSWER
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: checks for missing values

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [46]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [49]:
#ANSWER
bikes["windspeed"].isnull().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [50]:
#ANSWER
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [51]:
#ANSWER


Apply the Pandas `isna` function to the following data objects:

In [44]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [45]:
#ANSWER
print(pd.isna(x), pd.isna(y))

False True


In [52]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [53]:
#ANSWER
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: Converting coded strings to NaN

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [61]:
#ANSWER
bikes ['season'].value_counts()

3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [64]:
#ANSWER
print ("instant:", bikes['instant'].min(), "to" , bikes["instant"].max())
print ("dteday:", bikes['dteday'].min(), "to" , bikes["dteday"].max())
print ("windspeed:", bikes['windspeed'].min(), "to" , bikes["windspeed"].max())

instant: 1 to 17379
dteday: 2011-01-01 to 2012-12-31
windspeed: 0.0 to 0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [71]:
bikes_min, bikes_max = (min(bikes.min(numeric_only=True)), 
                        max(bikes.max(numeric_only=True)))
bikes_min, bikes_max

(0.0, 17379.0)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [0]:
bikes['atemp'].quantile(0.5)

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [74]:
#ANSWER
bikes['atemp'].quantile((0.1,0.25,0.5,0.75, 0.9))

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [76]:
type(bikes['atemp'])

pandas.core.series.Series

In [77]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
11999,12000,2012-05-20,2,1,5,4,0,0,0,1,0.52,0.5000,0.68,0.0896,2,14,16
2364,2365,2011-04-13,2,0,4,10,0,3,1,2,0.40,0.4091,0.87,0.1940,4,42,46
16120,16121,2012-11-09,4,1,11,10,0,5,1,1,0.36,0.3333,0.50,0.3284,34,163,197
5632,5633,2011-08-27,3,0,8,14,0,6,0,3,0.64,0.5758,0.89,0.5522,4,19,23
11455,11456,2012-04-27,2,1,4,12,0,5,1,1,0.46,0.4545,0.36,0.3582,63,244,307


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [75]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [79]:
#ANSWER
print(type(atemp_level))

<class 'pandas.core.series.Series'>


Here is a random sample of `atemp_level`:

In [80]:
atemp_level.sample(5)          

3130     (0.5, 0.75]
12205    (0.75, 1.0]
7552     (0.25, 0.5]
10364    (0.5, 0.75]
11552    (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [81]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

10897    mild
3826     warm
10511    mild
13578    warm
5251     warm
Name: atemp, dtype: category
Categories (4, object): [cool < mild < warm < hot]

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [84]:
#ANSWER
bikes['atemp_level']=atemp_level
bikes[(bikes.atemp_level=='mild') & (bikes.season==2)].season.count()

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [87]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [86]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.

In [116]:
rock = pd.read_csv('C:/Users/Travel Your Style/Documents/IOD Course 22 June 20/DATA/rock.csv')
print(rock)

                                       Song Clean ARTIST CLEAN Release Year  \
0                                Caught Up in You  .38 Special         1982   
1                                    Fantasy Girl  .38 Special          NaN   
2                                 Hold On Loosely  .38 Special         1981   
3                          Rockin' Into the Night  .38 Special         1980   
4                               Art For Arts Sake         10cc         1975   
...                                           ...          ...          ...   
2225                      She Loves My Automobile       ZZ Top          NaN   
2226                            Tube Snake Boogie       ZZ Top         1981   
2227                                         Tush       ZZ Top         1975   
2228                                   TV Dinners       ZZ Top         1983   
2229  WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO       ZZ Top         1973   

                                               COMB

## Check Column Names

Check column names and clean.

In [94]:

#check column names
rock.columns


Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

In [105]:
#clean
def clean_column_name(column_names):
    clean_column_names = []
    for c in column_names:
        c = c.lower().replace(' ', '_')
        c = c.lower().replace('*', '')
        c = c.lower().replace('?', '')
        clean_column_names.append(c)
        
    return clean_column_names

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [106]:
#check null values
rock.isnull().sum()

Song Clean        0
ARTIST CLEAN      0
Release Year    577
COMBINED          0
First?            0
Year?             0
PlayCount         0
F*G               0
dtype: int64

In [117]:
# Create a mask to find all rows with null values located in Release Year
null_mask = rock['release_year'].isnull()
rock.loc[null_mask, 'release_year'] = 0

KeyError: 'release_year'

In [112]:
#check null values changed
rock.isnull().sum()

Song Clean        0
ARTIST CLEAN      0
Release Year    577
COMBINED          0
First?            0
Year?             0
PlayCount         0
F*G               0
dtype: int64

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [114]:
rock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Song Clean    2230 non-null   object
 1   ARTIST CLEAN  2230 non-null   object
 2   Release Year  1653 non-null   object
 3   COMBINED      2230 non-null   object
 4   First?        2230 non-null   int64 
 5   Year?         2230 non-null   int64 
 6   PlayCount     2230 non-null   int64 
 7   F*G           2230 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


In [118]:
#Release Year should be an integer
rock['release_year'].value_counts()

KeyError: 'release_year'

In [119]:

# Making release year numeric
rock['release_year'] = pd.to_numeric(rock['release_year'])

KeyError: 'release_year'

# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [120]:
def check_min_max(df):
    # Check min, max of each column
    print(df.describe().T[['min', 'max']])

In [121]:
check_min_max(rock)

           min    max
First?     1.0    1.0
Year?      0.0    1.0
PlayCount  0.0  142.0
F*G        0.0  142.0


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [122]:
rock.dtypes


Song Clean      object
ARTIST CLEAN    object
Release Year    object
COMBINED        object
First?           int64
Year?            int64
PlayCount        int64
F*G              int64
dtype: object

In [126]:
def check_song(row):
    print('Song: ', row['song_clean'])
    print('Artist: ', row['artist_clean'])
    print('Released before 1970: ', row['release_year'] < 1970)
    print('\n')
    

In [124]:
rock.iloc[0]

Song Clean                     Caught Up in You
ARTIST CLEAN                        .38 Special
Release Year                               1982
COMBINED        Caught Up in You by .38 Special
First?                                        1
Year?                                         1
PlayCount                                    82
F*G                                          82
Name: 0, dtype: object

In [125]:
# Check any row using index
check_song(rock.iloc[0])

KeyError: 'song_clean'

In [129]:
# Check first five rows
rock.head(5).apply(check_song, axis=1)

KeyError: 'song_clean'

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [127]:

def convert_to_float(column):
    column = pd.to_numeric(column, errors='coerce')
    return column

## Apply these functions to your dataset

In [128]:
rock.apply(convert_to_float)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,NaN,NaN,1982.0,NaN,1,1,82,82
1,NaN,NaN,NaN,NaN,1,0,3,0
2,NaN,NaN,1981.0,NaN,1,1,85,85
3,NaN,NaN,1980.0,NaN,1,1,18,18
4,NaN,NaN,1975.0,NaN,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,NaN,NaN,NaN,NaN,1,0,1,0
2226,NaN,NaN,1981.0,NaN,1,1,32,32
2227,NaN,NaN,1975.0,NaN,1,1,109,109
2228,NaN,NaN,1983.0,NaN,1,1,1,1


## Describe the new float-only DataFrame.

In [136]:
rock.describe()


,First?,Year?,PlayCount,F*G
count,2230.0,2230.000000,2230.000000,2230.000000
mean,1.0,0.741256,16.872646,15.048430
std,0.0,0.438043,25.302972,25.288366
min,1.0,0.000000,0.000000,0.000000
25%,1.0,0.000000,1.000000,0.000000
50%,1.0,1.000000,4.000000,3.000000
75%,1.0,1.000000,21.000000,18.000000
max,1.0,1.000000,142.000000,142.000000




> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



